# Generate Queries from line names
* Author: Stefan Roland Schwingenschlögl
* email: stefan.roland.schwingenschloegl@gmail.com
* github: https://github.com/stefan-schwingenschloegl/
---
This notebooks is intended to generate the Kedro specific data catalog definition.

## Input:
The parameters for the kedro dataset definitions are stated in `parameters_API_database_ingestion.yml` and are:
<pre>
lines_to_evaluate: [line_1, line_2, ..., line_n]
wl_monitor_url: {endpoint to the Wiener Linien API}
line_directions_to_consider: [direction_1, direction_2]
</pre>

These parameters are defining which stopIDs are needed to be called.

## Output
The result of this notebook are the resulting kedro dataset entries. As a result the The structure of the resulting string is:
<pre>
API_{human_readable_line_name}_Call:
    type: api.APIDataset
    url: {Query to call all stops from the line}
</pre>

The ouptut needs to be copied and pasted into the `catalog.yml` -file. Because of this manual step in the analysis this user output is generated here in a notebook and not in a kedro pipeline.

In [1]:
from typing import List, Dict
import pandas as pd

# Pipeline Code to generate Query from line Name

In [2]:
def get_query():
    """Generate Query for the human readable line name and directions stated in the confgig file
    """
    # declare variables from config files
    line_names = catalog.load('params:lines_to_evaluate')
    wl_monitor_endpoint = catalog.load('params:wl_monitor_url')
    line_directions_to_consider = catalog.load("params:line_directions_to_consider")

    # get lineID from human readable name
    lineIDs = get_lineID(line_names)

    # get all stopID's from lineID
    stop_df = get_stopIDs(lineIDs, line_directions_to_consider)

    # generate query from information
    query_dict = generate_query_string(wl_monitor_endpoint, stop_df)

    # print all queries as kedro catalog entry
    print_data_in_kedro_catalog_format(query_dict)

def get_lineID(line_names: List[str]) -> str:
    """
    translate human readable line_name to API internal lineID

    :param line_name str: human known name of line to be assessed
    :return str: API internal lineID
    """
    lines = catalog.load("db_linie").drop_duplicates()

    lineIDs = lines.loc[lines['LineText'].isin(line_names)][['LineID', 'LineText']].drop_duplicates()

    return lineIDs

def get_stopIDs(lineIDs:pd.DataFrame,
                line_directions_to_consider: List[int]) -> pd.DataFrame:
    """
    Find all stopIDs to API lineID. The stops are the most common route per line

    :param lineID str: API internal lineID
    :return str: API internal stopIDs
    """
    routes = catalog.load("db_fahrwegverlaeufe").drop_duplicates()

    # prefilter before join to keep computational costs of join low
    routes = routes.loc[(routes['LineID'].isin(lineIDs['LineID'].unique())) & \
                        (routes['PatternID'].isin(line_directions_to_consider))]

    # join tables
    stop_df = lineIDs.merge(routes,
                  on = 'LineID',
                  how='left')

    return stop_df

def generate_query_string(wl_monitor_endpoint: str,
                          stop_df: pd.DataFrame) -> Dict[str, str]:
    """
    Generate dictionary with all query parameters per line as dictionary

    :param wl_monitor_endpoint str: endpoint of the Wiener Linien API -> defined in config
    :param stop_df pd.DataFrame: pandas dataframe with information 
    :return Dict[str, str]: Dictionary where the key is the line and the value is the query string
    """

    query_dict = {}
    
    for line in stop_df['LineText'].unique():
        stop_df_line = stop_df.loc[stop_df['LineText'] == line]
        query_dict[line] = wl_monitor_endpoint + 'StopID=' + '&StopID='.join(str(stop) for stop in stop_df_line['StopID'].unique())


    return query_dict

def print_data_in_kedro_catalog_format(query_dict: Dict[str, str]):
    """
    print all queries in kedro catalog format. The printed output can be copied and pasted into the catalog.yml-file

    :params query_dict Dict[str,str]: Dictionary where the key is the line and the value is the query string
    """
    print("\nHere are the Catalog Entries for a single API call for the specified Lines.\n"\
          "Copy these to clipboard and insert into catalog.yml in folder structure")
    for line in query_dict.keys():
        print(f"\n~~~ Kedro Catalog Entries for {line} ~~~")
        print(f"API_{line}_Call:")
        print(f"  type: api.APIDataset")
        print(f"  url: {query_dict[line]}")

        

get_query()

[12/27/23 13:49:34] INFO     Loading data from params:lines_to_evaluate (MemoryDataset)...      ]8;id=753069;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=557705;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\

                    INFO     Loading data from params:wl_monitor_url (MemoryDataset)...         ]8;id=481727;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=189535;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\

                    INFO     Loading data from params:line_directions_to_consider               ]8;id=21039;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=187025;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from db_linie (SQLQueryDataset)...                    ]8;id=6679;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=906906;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\

                    INFO     Loading data from db_fahrwegverlaeufe (SQLQueryDataset)...         ]8;id=867611;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=33283;file://C:\Users\sschw\miniconda3\envs\wl-data-engineering\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\


Here are the Catalog Entries for a single API call for the specified Lines.
Copy these to clipboard and insert into catalog.yml in folder structure

~~~ Kedro Catalog Entries for 1A ~~~
API_1A_Call:
  type: api.APIDataset
  url: http://www.wienerlinien.at/ogd_realtime/monitor?StopID=2915&StopID=2904&StopID=2905&StopID=2906&StopID=2907&StopID=2910&StopID=2911&StopID=2938&StopID=2926&StopID=2945&StopID=2914&StopID=2944
